In [ ]:
!pip install --quiet nltk
!pip install --quiet bs4
!pip install --quiet tqdm
!pip install --quiet scipy scikit-learn keras-tuner
!pip install --quiet keras-tuner

In [ ]:
import pandas as pd
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

## Loading in the data and preprocessing

In [ ]:
# Define the file paths
file_paths = [
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00000-of-00003.parquet',
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00001-of-00003.parquet',
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00002-of-00003.parquet',
]

# Load and concatenate the Parquet files
train_df_list = [pd.read_parquet(file) for file in file_paths]
train_df = pd.concat(train_df_list, ignore_index=True)

# Randomly sample 150,000 rows
sampled_df = train_df.sample(n=1000, random_state=1)

# Display the shape and the first few rows of the sampled DataFrame
print(sampled_df.shape)
print(sampled_df.head())


In [ ]:
sampled_df.to_csv('sampled_df.csv', index=False)

In [ ]:
lemmatizer = WordNetLemmatizer()

file_path = 'data.csv'
df = pd.read_csv(file_path)

# Define stop words
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words and lemmatize
    cleaned_text = ' '.join([lemmatizer.lemmatize(word) for word in words if word not in stop_words])
    
    return cleaned_text

# Apply the cleaning function with progress bars
tqdm.pandas()

In [ ]:
# Apply the cleaning function to both the 'article' and 'highlights' columns
df['cleaned_article'] = df['article'].progress_apply(clean_text)

In [ ]:
df['cleaned_highlights'] = df['highlights'].progress_apply(clean_text)

In [ ]:
cleaned_file_path = 'cleaned_data.csv'

In [ ]:
df.to_csv(cleaned_file_path, index=False)

## Loading in our preprocessed df

ubuntu commands required to load in the file uploaded to github using the large file system.
sudo apt-get update
sudo apt-get install git-lfs
git lfs install
git lfs pull


In [ ]:
df = pd.read_csv('cleaned_data.csv')
df.head()

In [ ]:
!pip install --quiet tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Mapping words onto numbers using the Tensorflow Tokenizer

What should the vocab_size be?

In [ ]:
combined_texts = df.cleaned_article.tolist() + df.cleaned_highlights.tolist()

In [37]:
from collections import Counter
def count_oov_in_text(tokenized_text, oov_token_index):
    return sum(1 for token in tokenized_text if token == oov_token_index)

def calculate_metrics(vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(combined_texts)
    
    # Tokenize articles and summaries with the tokenizer
    tokenized_articles = tokenizer.texts_to_sequences(df.cleaned_article)
    tokenized_summaries = tokenizer.texts_to_sequences(df.cleaned_highlights)
    
    word_counts = tokenizer.word_counts
    total_words = sum(word_counts.values())
    
    most_common_words = Counter(word_counts).most_common(vocab_size)
    coverage = sum([count for word, count in most_common_words]) / total_words
    
    oov_token_index = tokenizer.word_index.get('<OOV>')
    
    oov_counts_articles = [count_oov_in_text(text, oov_token_index) for text in tokenized_articles]
    oov_counts_summaries = [count_oov_in_text(text, oov_token_index) for text in tokenized_summaries]
    
    # OOV proportions
    total_count_articles = sum(len(seq) for seq in tokenized_articles)
    total_count_summaries = sum(len(seq) for seq in tokenized_summaries)
    oov_count_articles = sum(oov_counts_articles)
    oov_count_summaries = sum(oov_counts_summaries)
    
    oov_proportion_articles = oov_count_articles / total_count_articles
    oov_proportion_summaries = oov_count_summaries / total_count_summaries
    
    # Overall OOV proportion
    total_oov_count = oov_count_articles + oov_count_summaries
    total_token_count = total_count_articles + total_count_summaries
    overall_oov_proportion = total_oov_count / total_token_count
    
    avg_oov_articles = np.mean(oov_counts_articles)
    avg_oov_summaries = np.mean(oov_counts_summaries)
    
    return {
        "vocab_size": vocab_size,
        "coverage": coverage,
        "oov_proportion_articles": oov_proportion_articles,
        "oov_proportion_summaries": oov_proportion_summaries,
        "overall_oov_proportion": overall_oov_proportion,
        "avg_oov_articles": avg_oov_articles,
        "avg_oov_summaries": avg_oov_summaries
    }

vocab_sizes = [10000, 15000, 20000]

# Calculate metrics for each vocab size
results = []
for vocab_size in vocab_sizes:
    metrics = calculate_metrics(vocab_size)
    results.append(metrics)

results_df = pd.DataFrame(results)

In [38]:
results_df

,vocab_size,coverage,oov_proportion_articles,oov_proportion_summaries,overall_oov_proportion,avg_oov_articles,avg_oov_summaries
0,10000,0.884090,0.115653,0.119431,0.115929,43.896080,3.564533
1,15000,0.918553,0.081367,0.082596,0.081457,30.882813,2.465167
2,20000,0.937588,0.062432,0.062236,0.062418,23.696067,1.857480


In [ ]:
vocab_size = 16500
shared_tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
shared_tokenizer.fit_on_texts(combined_texts)

# Tokenize articles and summaries with the shared tokenizer
tokenized_article = shared_tokenizer.texts_to_sequences(df.cleaned_article)
tokenized_summary = shared_tokenizer.texts_to_sequences(df.cleaned_highlights)

### Potential outlier detection

In [ ]:
max_article_len = max(len(sequence) for sequence in tokenized_article)
max_summary_len = max(len(sequence) for sequence in tokenized_summary)

print(max_article_len, max_summary_len)

In [ ]:
import matplotlib.pyplot as plt
!pip install --quiet seaborn
import seaborn as sns

article_lengths = [len(sequence) for sequence in tokenized_article]
summary_lengths = [len(sequence) for sequence in tokenized_summary]

# Create a DataFrame for plotting
lengths_df = pd.DataFrame({
    'Article Length': article_lengths,
    'Summary Length': summary_lengths
})

# Melt the DataFrame to a long format
lengths_df_melted = lengths_df.melt(var_name='Type', value_name='Length')

# Plot violin plots using Seaborn
plt.figure(figsize=(10, 8))

# Violin plot for Article Lengths
plt.subplot(1, 2, 1)
sns.boxplot(y='Article Length', data=lengths_df, palette='Set2')
plt.title('Violin Plot of Article Lengths')
plt.xlabel('')
plt.ylabel('Length')

# Violin plot for Summary Lengths
plt.subplot(1, 2, 2)
sns.boxplot(y='Summary Length', data=lengths_df, palette='Set2')
plt.title('Box-plot of Summary Lengths')
plt.xlabel('')
plt.ylabel('Length')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate IQR for article lengths
article_q1 = np.percentile(article_lengths, 25)
article_q3 = np.percentile(article_lengths, 75)
article_iqr = article_q3 - article_q1

# Calculate IQR for summary lengths
summary_q1 = np.percentile(summary_lengths, 25)
summary_q3 = np.percentile(summary_lengths, 75)
summary_iqr = summary_q3 - summary_q1

In [ ]:
# Identify indices of articles outside the IQR and below 50 tokens
indices_to_remove_articles = [i for i, length in enumerate(article_lengths) 
                              if length < 50 or length < (article_q1 - 1.5 * article_iqr) or length > (article_q3 + 1.5 * article_iqr)]
print("Instances removed (articles): ", len(indices_to_remove_articles))

# Remove the corresponding articles and summaries based on article criteria
filtered_articles = [article for i, article in enumerate(tokenized_article) if i not in indices_to_remove_articles]
filtered_summaries = [summary for i, summary in enumerate(tokenized_summary) if i not in indices_to_remove_articles]


In [ ]:
# Recalculate lengths of filtered summaries
filtered_summary_lengths = [len(sequence) for sequence in filtered_summaries]

# Identify indices of summaries outside the IQR and below 5 tokens
indices_to_remove_summaries = [i for i, length in enumerate(filtered_summary_lengths) 
                               if length < 5 or length < (summary_q1 - 1.5 * summary_iqr) or length > (summary_q3 + 1.5 * summary_iqr)]
print("Instances removed (summaries): ", len(indices_to_remove_summaries))

# Remove the corresponding articles and summaries based on summary criteria
filtered_articles = [article for i, article in enumerate(filtered_articles) if i not in indices_to_remove_summaries]
filtered_summaries = [summary for i, summary in enumerate(filtered_summaries) if i not in indices_to_remove_summaries]

# Print the number of remaining instances
print("Remaining instances: ", len(filtered_articles))

In [ ]:
print(len(filtered_articles), len(filtered_summaries))

## Padding to ensure that our text-sequences are of the same length

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_article_sequences = pad_sequences(filtered_articles, max_article_len, padding='post', truncating='post')
padded_summary_sequences = pad_sequences(filtered_summaries, max_summary_len, padding = 'post', truncating = 'post')

## Building our network

In [ ]:
!pip install --quiet scikit-learn
from sklearn.model_selection import train_test_split
# Create decoder input sequences by right-shifting the padded summary sequences
decoder_input_sequences = np.zeros_like(padded_summary_sequences)
decoder_input_sequences[:, 1:] = padded_summary_sequences[:, :-1]
decoder_input_sequences[:, 0] = 0  # Use padding token (0) as the start token

labels = padded_summary_sequences

# Combine encoder inputs, decoder inputs, and labels for consistent splitting
data_combined = list(zip(padded_article_sequences, decoder_input_sequences))
labels_combined = labels

# Split the data into training (125,000) and test sets (15,000)
X_train_combined, X_test_combined, y_train, y_test = train_test_split(data_combined, labels_combined, test_size=15000, random_state=42)
# Split the data with indices
data_with_indices = list(enumerate(data_combined))
X_train_combined_with_indices, X_test_combined_with_indices, y_train, y_test = train_test_split(data_with_indices, labels_combined, test_size=15000, random_state=42)

# Extract indices of the test set
test_indices = [index for index, _ in X_test_combined_with_indices]

# Further split the training data into training and validation sets (80-20 split of 125,000)
X_train_combined, X_val_combined, y_train, y_val = train_test_split(X_train_combined, y_train, test_size=0.2, random_state=42)

# Separate encoder and decoder inputs after splitting
X_train_encoder, X_train_decoder = zip(*X_train_combined)
X_val_encoder, X_val_decoder = zip(*X_val_combined)
X_test_encoder, X_test_decoder = zip(*X_test_combined)

# Convert tuples back to numpy arrays
X_train_encoder = np.array(X_train_encoder)
X_train_decoder = np.array(X_train_decoder)
X_val_encoder = np.array(X_val_encoder)
X_val_decoder = np.array(X_val_decoder)
X_test_encoder = np.array(X_test_encoder)
X_test_decoder = np.array(X_test_decoder)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Fixed hyperparameters
lstm_units = 150  
embedding_dim = 250  
dropout_rate = 0.2  


# Build the model
def build_model():
    # Encoder
    encoder_inputs = Input(shape=(max_article_len,))
    encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(units=lstm_units, return_state=True, dropout=dropout_rate)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(max_summary_len,))
    decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(units=lstm_units, return_sequences=True, return_state=True, dropout=dropout_rate)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    # Compile the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    optimizer = Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
model = build_model()

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True, min_delta=0.01)

# Define model checkpoint callback
checkpoint = ModelCheckpoint('best_model2.keras', monitor='val_loss', save_best_only=True, mode='min')

# Train the model with early stopping and model checkpoint
history = model.fit(
    [X_train_encoder, X_train_decoder], y_train,
    epochs=5,
    validation_data=([X_val_encoder, X_val_decoder], y_val),
    batch_size=170,
    callbacks=[early_stopping, checkpoint]
)
model.save('final_model.keras')

In [ ]:
model.save('model2.keras')

## Bart

In [14]:
!pip install --quiet --upgrade transformers sentencepiece datasets torch accelerate

import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline
from datasets import load_metric

In [ ]:
!pip install --quiet tf-keras

In [62]:
# Initialize the BART summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")

# Use the same test indices to filter the original DataFrame
test_df = df.iloc[test_indices].reset_index(drop=True)

In [67]:
reference_summaries = test_df.cleaned_highlights
df_test = pd.DataFrame(test_df.cleaned_article)

In [69]:
!pip install --quiet tqdm
from tqdm import tqdm
tqdm.pandas()
# Generate summaries using the BART model with progress bar
def generate_summary(text, summarizer):
    try:
        input_length = len(summarizer.tokenizer.tokenize(text))
        max_length = min(max(50, input_length // 2), 512)  # Ensure max_length is within reasonable limits and less than input_length
        return summarizer(text, max_length=max_length, min_length=30, do_sample=False)[0]['summary_text']
    except Exception as e:
        print(f"Error processing article: {e}")
        return ""

df_test['bart_summary'] = df_test['cleaned_article'].progress_apply(lambda x: generate_summary(x, summarizer))


  0%|          | 9/15000 [00:24<11:57:44,  2.87s/it]

Error processing article: index out of range in self


  1%|          | 94/15000 [04:25<12:05:48,  2.92s/it]

Error processing article: index out of range in self


  1%|          | 120/15000 [05:35<11:48:21,  2.86s/it]

...

100%|█████████▉| 14925/15000 [12:02:14<03:45,  3.01s/it]

Error processing article: index out of range in self


100%|█████████▉| 14992/15000 [12:05:59<00:23,  2.91s/it]

Error processing article: index out of range in self


100%|██████████| 15000/15000 [12:06:22<00:00,  2.91s/it]


In [70]:
# Export the DataFrame to a CSV file
df_test.to_csv('df_test_with_summaries.csv', index=False)

In [ ]:
!pip install --quiet evaluate
import evaluate

# Function to calculate and print ROUGE scores for the first few entries
def calculate_rouge(df):
    for i in range(len(df)):
        reference = df.iloc[i]['cleaned_summaries']
        hypothesis = df.iloc[i]['bart_summary']
        scores = rouge.compute(predictions=[hypothesis], references=[reference])
        print(f"Article {i+1} ROUGE scores:")
        print(scores)
        print("\n" + "="*80 + "\n")

calculate_rouge(df_w_summaries)

Article 1 ROUGE scores:
{'rouge1': AggregateScore(low=Score(precision=0.21818181818181817, recall=0.5454545454545454, fmeasure=0.3116883116883117), mid=Score(precision=0.21818181818181817, recall=0.5454545454545454, fmeasure=0.3116883116883117), high=Score(precision=0.21818181818181817, recall=0.5454545454545454, fmeasure=0.3116883116883117)), 'rouge2': AggregateScore(low=Score(precision=0.037037037037037035, recall=0.09523809523809523, fmeasure=0.05333333333333334), mid=Score(precision=0.037037037037037035, recall=0.09523809523809523, fmeasure=0.05333333333333334), high=Score(precision=0.037037037037037035, recall=0.09523809523809523, fmeasure=0.05333333333333334)), 'rougeL': AggregateScore(low=Score(precision=0.14545454545454545, recall=0.36363636363636365, fmeasure=0.2077922077922078), mid=Score(precision=0.14545454545454545, recall=0.36363636363636365, fmeasure=0.2077922077922078), high=Score(precision=0.14545454545454545, recall=0.36363636363636365, fmeasure=0.2077922077922078)), 

In [16]:
df_w_summaries = pd.read_csv('df_w_summaries.csv')
df_w_summaries.head()

,cleaned_article,bart_summary,cleaned_summaries
0,police officer pulled shoplifter decided help ...,Police officer pulled shoplifter decided help ...,justin roby london pd kentucky called shoplift...
1,cnn billed astronomical equivalent large hadro...,Ska international effort build world largest r...,world largest radio telescope uncover secret h...
2,expert discovered mount snowdon whole metre ta...,mount snowdon whole metre taller originally t...,snowdon one metre taller recognised ordnance s...
3,lucy waterlow published est july updated est j...,Teenager cramlington northumberland went missi...,teenager appeared morning mother rhonda said h...
4,man armed kitchen knife rugbytackled ground po...,David belmar haringey north london admitted tr...,david belmar admitted trespass possession blad...


In [19]:
import difflib

def highlight_differences(text1, text2):
    d = difflib.Differ()
    diff = d.compare(text1.split(), text2.split())
    return '\n'.join(diff)

# Display differences for the first few entries
for i in range(5):
    print(f"Article {i+1}:\n{df_w_summaries.iloc[i]['cleaned_article']}\n")
    print(f"BART Summary:\n{df_w_summaries.iloc[i]['bart_summary']}\n")
    print(f"Reference Summary:\n{df_w_summaries.iloc[i]['cleaned_summaries']}\n")
    print("Differences:")
    print(highlight_differences(df_w_summaries.iloc[i]['bart_summary'], df_w_summaries.iloc[i]['cleaned_summaries']))
    print("\n" + "="*80 + "\n")


Article 1:
police officer pulled shoplifter decided help alleged criminal saw single father trying feed sixmonthold son justin roby detective london police department kentucky called incident supermarket saturday january refused detain man instead bought formula needed let walk free scroll video kind justin roby detective london police department kentucky bought shoplifter baby formula citing court wouldnt done good roby told wkyt he already short money cant afford formula making appear court he still going food baby police officer black white u said officer justin roby there lot gray cipher everything really need figure whole story put situation said black white roby insists police officer show discretion situation help people think well son u think lot people look u see uniform car tool belt roby said behind uniform im human im person community like little boy im father like gentleman robot said there human behind badge

BART Summary:
Police officer pulled shoplifter decided help all

In [18]:
import difflib

def highlight_differences(text1, text2):
    d = difflib.Differ()
    diff = d.compare(text1.split(), text2.split())
    return '\n'.join(diff)

# Example comparison
for i in range(len(df)):
    print(f"Article {i+1}:\n{df.iloc[i]['cleaned_article']}\n")
    print(f"BART Summary:\n{df.iloc[i]['bart_summary']}\n")
    print(f"Reference Summary:\n{df.iloc[i]['cleaned_summaries']}\n")
    print("Differences:")
    print(highlight_differences(df.iloc[i]['bart_summary'], df.iloc[i]['cleaned_summaries']))
    print("\n" + "="*80 + "\n")


KeyError: 'cleaned_article'